# End Child Poverty Act

A simulation of the End Child Poverty Act and its possible effects (TODO: make a nice paragraph describing what we're doing here)

In [1]:
from openfisca_us import Microsimulation, IndividualSim

In [2]:
from openfisca_us.model_api import *
from openfisca_us.tools.baseline_variables import baseline_variables

REPLACED_PROGRAMS = ["ctc", "eitc"]

CHILD_AMOUNT = 393
CHILD_AGE_LIMIT = 18
ADULT_DEPENDENT_CREDIT = 600
FILER_CREDIT_SINGLE = 600
FILER_CREDIT_MARRIED = 1200
FILER_CREDIT_SINGLE_PHASE_OUT_START = 20_000
FILER_CREDIT_MARRIED_PHASE_OUT_START = 40_000
FILER_CREDIT_PHASE_OUT_RATE = 0.05
FILER_CREDIT_MIN_AGE = 19
FILER_CREDIT_MAX_AGE = 64

class ecpa(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR
    label = "End Child Poverty Act"

    def formula(tax_unit, period, parameters):
        person = tax_unit.members
        # Child allowance.
        qualifies_for_child_allowance = person("age", period) <= CHILD_AGE_LIMIT
        child_allowance = CHILD_AMOUNT * tax_unit.sum(qualifies_for_child_allowance) * 12
        # Adult dependent credit.
        adult_dependent = person("is_tax_unit_dependent", period) & ~qualifies_for_child_allowance
        adult_dependent_credit = ADULT_DEPENDENT_CREDIT * tax_unit.sum(adult_dependent)
        # Filer credit.
        # Define eligibility based on age.
        age_head = tax_unit("age_head", period)
        age_spouse = tax_unit("age_spouse", period)
        head_qualifies = (age_head >= FILER_CREDIT_MIN_AGE) & (age_head <= FILER_CREDIT_MAX_AGE)
        spouse_qualifies = (age_spouse >= FILER_CREDIT_MIN_AGE) & (age_spouse <= FILER_CREDIT_MAX_AGE)
        filer_credit_eligible = head_qualifies | spouse_qualifies
        # Get maximum amount.
        filing_status = tax_unit("filing_status", period)
        is_married = filing_status == filing_status.possible_values.JOINT
        max_filer_credit = where(is_married, FILER_CREDIT_MARRIED, FILER_CREDIT_SINGLE)
        # Phase out.
        agi = tax_unit("adjusted_gross_income", period)
        phase_out_start = where(is_married, FILER_CREDIT_MARRIED_PHASE_OUT_START, FILER_CREDIT_SINGLE_PHASE_OUT_START)
        excess = max_(agi - phase_out_start, 0)
        reduction = excess * FILER_CREDIT_PHASE_OUT_RATE
        # Compute final amount.
        filer_credit = filer_credit_eligible * max_(max_filer_credit - reduction, 0)
        # Return all three components.
        return child_allowance + adult_dependent_credit + filer_credit

class spm_unit_net_income(Variable):
    def formula(spm_unit, period, parameters):
        original = baseline_variables["spm_unit_net_income"].formula(spm_unit, period, parameters)
        return original + add(spm_unit, period, ["ecpa"])


class add_ecpa(Reform):
    def apply(self):
        for program in REPLACED_PROGRAMS:
            self.neutralize_variable(program)
        self.add_variable(ecpa)
        self.update_variable(spm_unit_net_income)

In [3]:
isb = IndividualSim(year=2022)
isr = IndividualSim(add_ecpa, year=2022)

for individual_sim in [isb, isr]:
    individual_sim.add_person(name="parent1", age=25)
    individual_sim.add_person(name="parent2", age=25)
    individual_sim.add_person(name="child1", age=5)
    individual_sim.add_person(name="child2", age=5)
    members = ["parent1", "parent2", "child1", "child2"]
    # members = ["parent1", "child1", "child2"]
    individual_sim.add_tax_unit(
        name="tax_unit",
        members=members,
        premium_tax_credit=0,
    )
    individual_sim.add_spm_unit(
        name="spm_unit",
        members=members,
        snap_emergency_allotment=0,
    )
    individual_sim.add_household(
        name="household",
        members=members,
        state_code="CA",
    )
    individual_sim.vary("employment_income")

In [4]:
import pandas as pd

df = pd.DataFrame(
    dict(
        employment_income=isb.calc("employment_income")[0],
        net_income_baseline=isb.calc("spm_unit_net_income")[0],
        net_income_ecpa=isr.calc("spm_unit_net_income")[0],
        mtr_baseline=1 - isb.deriv("spm_unit_net_income", "employment_income", wrt_target="parent1"),
        mtr_ecpa=1 - isr.deriv("spm_unit_net_income", "employment_income", wrt_target="parent1"),
    )
)

df["diff"] = df["net_income_ecpa"] - df["net_income_baseline"]
df["mtr_diff"] = df["mtr_ecpa"] - df["mtr_baseline"]

In [5]:
import plotly.express as px

px.line(df, x="employment_income", y=["net_income_baseline", "net_income_ecpa"])

In [6]:
fig = px.line(df, x="employment_income", y="diff")
fig.update_layout(yaxis_range=[0, 11_000])
fig.show()

In [7]:
fig = px.line(
    df,
    x="employment_income",
    y=["mtr_baseline", "mtr_ecpa"]
)
fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat=".1%",
    yaxis_range=[-1, 1],
)
fig.show()

In [8]:
px.line(df, x="employment_income", y="mtr_diff")

## Microsimulation

In [9]:
from policyengine.countries import US

us = US()

mb, mr = us._get_microsimulations(add_ecpa)

In [17]:
res = us.population_reform(add_ecpa)

In [19]:
res.keys()

dict_keys(['net_cost', 'net_cost_numeric', 'poverty_change', 'winner_share', 'loser_share', 'gini_change', 'rel_income_decile_chart', 'avg_income_decile_chart', 'rel_wealth_decile_chart', 'avg_wealth_decile_chart', 'poverty_chart', 'deep_poverty_chart', 'waterfall_chart', 'intra_income_decile_chart', 'intra_wealth_decile_chart', 'inequality_chart'])

In [20]:
import plotly.graph_objects as go

go.Figure(data=res["rel_income_decile_chart"]["data"], layout=res["rel_income_decile_chart"]["layout"])

In [21]:
go.Figure(data=res["intra_income_decile_chart"]["data"], layout=res["intra_income_decile_chart"]["layout"])

In [22]:
go.Figure(data=res["poverty_chart"]["data"], layout=res["poverty_chart"]["layout"])

In [15]:
in_poverty = mr.calc("spm_unit_is_in_spm_poverty", map_to="person")
child = mb.calc("is_child")
child_poverty = (in_poverty & child).sum() / child.sum()

In [16]:
in_poverty

        value       weight
0       False  1530.790039
1       False  1761.680054
2       False  1418.760010
3       False  1406.530029
4       False  1489.829956
...       ...          ...
163538  False     0.000000
163539  False     0.000000
163540  False     0.000000
163541  False     0.000000
163542  False     0.000000

[163543 rows x 2 columns]